<a href="https://colab.research.google.com/github/iguv221/Tobigs_Assignment/blob/main/2%EC%A3%BC%EC%B0%A8_%ED%9A%8C%EA%B7%803_%EA%B9%80%EC%84%B1%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data 

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보 
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ToBigs/과제 폴더/2주차/18기 2주차 1 Regression 강의자료/과제/assignment3_creditcard.csv")

In [ ]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [ ]:
x = data.drop(columns='Class')
y = data['Class']

In [ ]:
### 0 과 1 의 비율 확인.
zero, one = y.value_counts()
print(zero,one)
print(one/len(y))

28432 246
0.008578004044912477


# 정확도 말고 precision 과 recall 을 개선해야함, 아니면 그 둘의 조화평균인 f score 을 개선하자.
# 정확도를 이용하면 당연히 높게 나올 수 밖에 없다. 왜냐하면 모델이 전부 다 0 으로 예상해도 대충 99.2%의 정확도를 보일 것이다. 하지만 0보다 훨 중요한 1을 하나도 예측 못하는 꼴이 난다. 이러한 상황을 방지하기 위해 f score 을 사용한다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
scaled_x = pd.DataFrame(scaler.transform(x))

In [ ]:
scaled_x

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.871094,0.700152,0.895083,0.377896,0.550269,0.550335,0.472779,0.490361,0.553626,0.559270,...,0.485285,0.503374,0.332166,0.565040,0.801008,0.331685,0.838410,0.300850,0.398883,0.403262
1,0.990056,0.655167,0.843226,0.286572,0.561661,0.480156,0.530337,0.658296,0.677860,0.606995,...,0.534559,0.447455,0.441802,0.537836,0.792766,0.497511,0.738524,0.600501,0.394815,0.389143
2,0.836587,0.619505,0.918240,0.317772,0.546192,0.487324,0.537965,0.673189,0.643824,0.570059,...,0.379143,0.477336,0.448603,0.492854,0.817732,0.468829,0.728680,0.559988,0.383724,0.379434
3,0.882285,0.686921,0.892562,0.365873,0.554476,0.477110,0.537083,0.671662,0.587236,0.623826,...,0.536430,0.449884,0.443322,0.538731,0.789353,0.488938,0.704320,0.340402,0.401302,0.385691
4,0.918625,0.657811,0.822817,0.269253,0.562942,0.459875,0.555584,0.648243,0.565493,0.642868,...,0.415059,0.430694,0.440835,0.574069,0.791290,0.419307,0.568556,0.558000,0.426611,0.405300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28673,0.989798,0.662913,0.854663,0.341371,0.560597,0.466295,0.534000,0.655969,0.642438,0.616840,...,0.482978,0.445514,0.435189,0.478949,0.799022,0.418271,0.698556,0.453759,0.396341,0.389450
28674,0.965567,0.643371,0.929186,0.306251,0.535430,0.505545,0.508323,0.670401,0.669983,0.622545,...,0.534421,0.446445,0.443900,0.556880,0.791036,0.423296,0.764795,0.416044,0.404679,0.392624
28675,0.990082,0.665663,0.834263,0.339613,0.568946,0.475936,0.534044,0.657834,0.641151,0.605412,...,0.479624,0.447311,0.433498,0.462417,0.798441,0.467971,0.704778,0.451027,0.396662,0.390633
28676,0.906223,0.683769,0.926884,0.435159,0.575592,0.508484,0.541827,0.654583,0.591839,0.653109,...,0.474869,0.449726,0.443202,0.541485,0.784193,0.292068,0.688941,0.436758,0.354079,0.379065


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, stratify = y, test_size=0.1, random_state = 42)

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
### 과적합 유무를 알아보기 위해 train 과 test 둘의 성능을 확인한다.
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [ ]:
def f1score_accuracy(Real,Pred) :
    Pred = np.array(Pred)
    y = np.array(Real)
    TP = np.sum((Pred == 1) & (y == 1))
    FP = np.sum((Pred == 1) & (y != 1))
    FN = np.sum((Pred != 1) & (y == 1))
    TN = np.sum((Pred != 1) & (y != 1))
    Precision = (TP)/(TP+FP)
    Recall = (TP)/(TP+FN)
    f1_score = 2*(Precision*Recall)/(Precision+Recall)
    accuracy = (TP+TN)/(TP+FP+TN+FN)
    return round(f1_score,4), round(accuracy,4)

In [ ]:
print(f1score_accuracy(y_train,train_pred))
print(f1score_accuracy(y_test,test_pred))

(0.7751, 0.9968)
(0.7805, 0.9969)


## 다행히 f1score 이 train 과 test 끼리 큰 차이를 안 보이기 때문에 과적합은 안 일어났다고 할 수 있다. 